In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from torch.optim import lr_scheduler
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from distutils.dir_util import copy_tree
import random
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
### Process dataset, put dog images of the same breed in the same folder

PATH='/kaggle/input/dog-breed-identification/'
W_PATH='/kaggle/working/dog-breed-identification/'
labels = pd.read_csv(f'{PATH}labels.csv')

os.mkdir(f'/kaggle/working/dog-breed-identification')
os.mkdir(f'{W_PATH}valid')
os.mkdir(f'{W_PATH}train')
os.mkdir(f'{W_PATH}test')
os.mkdir(f'{W_PATH}test/0')  # for allowing ImageFolder to work

for f in labels.breed.unique():
    os.mkdir(f'{W_PATH}train/{f}')
    os.mkdir(f'{W_PATH}valid/{f}')

n = 0.1  # portion of validation set

# Move the train and test data from input to working directory
copy_tree(f'{PATH}train', f'{W_PATH}train')
copy_tree(f'{PATH}test', f'{W_PATH}test/0')

# Move the training data to subfolders by their labels
for p in labels.itertuples():
    file_path = f'{W_PATH}train/{p.id}.jpg'
    train_path = f'{W_PATH}train/{p.breed}/{p.id}.jpg'
    os.rename(f'{file_path}', f'{train_path}')

# Move a portion of validation data from training folders to validation folders
for f in os.listdir(f'{W_PATH}train'):
    pics = os.listdir(f'{W_PATH}train/{f}')
    num_pics = len(pics)
    num_val_pics = round(n * num_pics)
    val_pics = random.sample(pics, num_val_pics)
    
    for p in val_pics:
        file_path = f'{W_PATH}train/{f}/{p}'
        val_path = f'{W_PATH}valid/{f}/{p}'
        os.rename(f'{file_path}', f'{val_path}')


In [ ]:
def get_dogs_data(augmentation=0):
    transform_train = transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    transform_test = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    trainset = torchvision.datasets.ImageFolder(root=f'{W_PATH}train', transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
    
    validset = torchvision.datasets.ImageFolder(root=f'{W_PATH}valid', transform=transform_test)
    validloader = torch.utils.data.DataLoader(validset, batch_size=128, shuffle=False, num_workers=2)
    
    testset = torchvision.datasets.ImageFolder(root=f'{W_PATH}test', transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)
    
    return {'train': trainloader, 'valid': validloader, 'test': testloader, 'classes': trainloader.dataset.classes}

data = get_dogs_data()

In [ ]:
### Visualization

def smooth(x, size):
  return np.convolve(x, np.ones(size)/size, mode='valid')

dataiter = iter(data['train'])
images, labels = dataiter.next()
images = images[:8]
print(images.size())

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print("Labels:" + ', '.join('%9s' % data['classes'][labels[j]] for j in range(8)))

In [ ]:
### Method for training the neural network model
### AUTHOR Yixiao Li


def train_model(model, data, optimizer, scheduler, loss_func, num_epochs=5):
    """
    Trains the model on train set and evaluates on validation set

    Arguments:
        model is the model to be trained and evaluated
        data is a dictionary that contains mappings for the train and validation set
        optimizer is used to optimize the model during training
        loss_func is the loss function
        num_epochs is a positive integer

    Returns:
        the loss on train set over epochs
        the loss on validation set over epochs
        the highest validation accuracy
        the model with the highest validation accuracy
    """
    train_losses = []
    valid_losses = []
    best_valid_accuracy = 0.0
    best_model_state = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        print(epoch + 1, "/", num_epochs, "epochs")  # keep track of progress

        # there is a training phase and validation phase for each epoch
        for phase in ['train', 'valid']:                
            if phase == 'train':
                model = model.train()  # set model to training mode
            else:
                model = model.eval()   # set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # iterate over the dataset in batches using dataloader
            for inputs, labels in data[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward; track history only if training
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, predicts = torch.max(outputs, 1)
                    loss = loss_func(outputs, labels)

                    # backward + optimize only if training
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # add iteration statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(predicts == labels.data)

            if phase == 'train':
                scheduler.step()
            
            # epoch statistics
            num_data = len(data[phase].dataset)
            epoch_loss = running_loss / num_data
            epoch_accuracy = running_corrects.double() / num_data

            # record epoch loss
            if phase == 'train':
                print('train loss: ', epoch_loss)
                train_losses.append(epoch_loss)
            else:
                print('validation loss: ', epoch_loss)
                valid_losses.append(epoch_loss)

            # deep copy the model only if validating and model has highest validation accuracy
            if phase == 'valid' and epoch_accuracy > best_valid_accuracy:
                best_valid_accuracy = epoch_accuracy
                best_model_state = copy.deepcopy(model.state_dict())

    # load best model
    model.load_state_dict(best_model_state)

    return train_losses, valid_losses, best_valid_accuracy, model


In [ ]:
### Configurations

# number of classes
num_classes = 120

# initialize for transfer learning
fine_tune = False

### model
#model = torch.hub.load('pytorch/vision:v0.6.0', 'inception_v3', pretrained=True, aux_logits=False)
#model = model = torch.hub.load('pytorch/vision:v0.6.0', 'resnext50_32x4d', pretrained=True)
model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x8d_wsl')
#model = torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models', 'resnext101_32x8d_swsl')

if not fine_tune:
    for param in model.parameters():
        param.requires_grad = False
    model.fc.requires_grad = True
model.fc = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )
model = model.to(device)

# loss function
loss_func = torch.nn.functional.cross_entropy

# learning rate
lr = 0.003

# weight decay
weight_decay = 0.0005

# optimizer
optimizer = None
if not fine_tune:
    optimizer = torch.optim.Adam(model.fc.parameters(), lr=lr, weight_decay=weight_decay)
else:
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# step_size
step_size = 7

# gamma
gamma = 0.3

# scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

In [ ]:
### Training

# number of epochs
num_epochs = 32

# output important configurations
print('learning rate ', lr, ' weight decay ', weight_decay)

# train model
train_loss, valid_loss, highest_accuracy, model = train_model(model, data, optimizer, scheduler,
                                                                 loss_func, num_epochs)

# plot
fig, ax = plt.subplots()
ax.plot(train_loss, label="train", color="blue")
ax.plot(valid_loss, label="validation", color="red")
ax.set_title("Cross Entropy Loss vs. Epochs Graph")
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")
ax.legend()

plt.show()

print("Highest accuracy on validation set is ", highest_accuracy.item())
print()


In [ ]:
### Save the best model state

model_state_path = '../working/model/'
if not os.path.exists(model_state_path):
    os.makedirs(model_state_path)

state = model.state_dict()
torch.save(state, model_state_path + 'optimal_state')

In [ ]:
from IPython.display import FileLink
FileLink(r'model/optimal_state')

In [ ]:
### Make predictions on test data samples 
### (From https://www.kaggle.com/pjreddie/transfer-learning-to-birds-with-resnet18/notebook)

def predict(net, dataloader, ofname):
    out = open(ofname, 'w')
    out.write("path,class\n")
    net.to(device)
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (images, labels) in enumerate(dataloader, 0):
            if i%100 == 0:
                print(i)
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            fname, _ = dataloader.dataset.samples[i]
            out.write("test/{},{}\n".format(fname.split('/')[-1], data['classes'][predicted.item()]))
    out.close()


In [ ]:
predict(model, data['test'], f'{W_PATH}preds.csv')